In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.delete_all_components()

In [30]:
design.get_chip_z('Q_chip')

'5um'

In [3]:
design.chips['C_chip'].size['size_x']='8mm'
design.chips['C_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['size_x']='8mm'
design.chips['Q_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['center_z']='20um'

# Resonator length

In [4]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [5]:
resonator_length_2=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,2)
    a=a.strip('mm')
    a=float(a)-0.8-0.2-0.12 
    resonator_length_2.append(a)
print(resonator_length_2)


[7.333254683870284, 7.104788339841421, 6.88834654020913]


# Flipchip

In [6]:
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))

# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='Q_chip',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip'))]

qubits=q0

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-60}um",
                                              pos_y=f"{q0y+80}um",
                                              pad_width="150um",
                                              pad_height="504um",
                                              cpw_stub_height="200um",
                                              orientation='270',                                            
                                              chip="C_chip"))

design.rebuild()
gui.rebuild()
gui.autoscale()

In [7]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-3000um',
                                         pos_y = '2000um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '3000um',
                                         pos_y = '2000um',
                                         orientation='180',
                                         lead_length = '30um'))

In [8]:
clt0 = CoupledLineTee(design, "clt0", 
                        options=dict(pos_x='-2000um',
                                    pos_y='2000um', 
                                    coupling_length='300um',
                                    chip="C_chip",
                                    down_length='300um',
                                    fillet='99um',
                                    coupling_space='5um',
                                    mirror=True,
                                    open_termination=True))



In [9]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_start'))))
fl1=RouteStraight(design, 'fl1',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [10]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [11]:
rr0=connect('rr0', 'clt0', 'second_end', 'c0', 'in',
            resonator_length_2[1],jogs,jogs, 
            '400um','0um', '100um', '100um')

# EPR Analysis

In [12]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [13]:
hfss.start()

INFO 11:30AM [connect_project]: Connecting to Ansys Desktop API...
INFO 11:30AM [load_ansys_project]: 	Opened Ansys App
INFO 11:30AM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 11:30AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project20
INFO 11:30AM [connect_design]: 	Opened active design
	Design:    flip_V8 [Solution type: Eigenmode]
INFO 11:30AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 11:30AM [connect]: 	Connected to project "Project20" and design "flip_V8" 😀 



True

In [14]:
hfss.activate_ansys_design("flip_V8", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','clt0','rr0','fl0','fl1'],
                    [('clt0','prime_end')],
                    box_plus_buffer=False)

INFO 11:31AM [connect_design]: 	Opened active design
	Design:    flip_V8 [Solution type: Eigenmode]


In [15]:
hfss.close()

True

# LOM Analysis

In [16]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")


In [17]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.77',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.77',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [19]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 06:33PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp_3_u0p5w.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5770000000, Maxwell, 1, False
INFO 06:33PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp5czelae4.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5770000000, Maxwell, 1, False
INFO 06:33PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpol61mlei.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5770000000, Maxwell, 2, False
INFO 06:33PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpmqlfkld3.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5770000000, Maxwell, 3, False
INFO 06:33PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp3zpmdw6l.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5770000000, Maxwell, 4, False
INFO 06:33

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,402.58120,-72.91369,-48.75885,-221.29010
pad_bot_Q0,-72.91369,107.58240,-31.58072,-1.79317
pad_top_Q0,-48.75885,-31.58072,107.16900,-25.55200
quad_coupler_c0,-221.29010,-1.79317,-25.55200,249.99450


In [20]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 6.70
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.413042 [GHz]
EC 273.454503 [MHz]
EJ 14.854145 [GHz]
alpha -311.873615 [MHz]
dispersion 5.157754 [KHz]
Lq 10.995564 [nH]
Cq 70.835287 [fF]
T1 18.397663 [us]

**Coupling Properties**

tCqbus1 -11.915752 [fF]
gbus1_in_MHz -163.620421 [MHz]
χ_bus1 -8.232341 [MHz]
1/T1bus1 8650.824021 [Hz]
T1bus1 18.397663 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.719986,307.449927,14.854145,-354.199047,16.226989,[-116.96867088425435],[-7.4770835378939475],7.477084,116.968671
2,5.611213,295.144279,14.854145,-338.792179,10.969067,[-230.19549930588073],[-23.34368271260597],23.343683,230.195499
3,5.581568,291.839997,14.854145,-334.671808,9.832215,[-227.75517188816045],[-21.60047556621049],21.600476,227.755172
4,5.588509,292.611807,14.854145,-335.633609,10.088435,[-197.76454225556844],[-16.501010876523097],16.501011,197.764542
5,5.530484,286.195466,14.854145,-327.649447,8.119934,[-179.9151333854487],[-12.260027221335127],12.260027,179.915133
6,5.479858,280.662992,14.854145,-320.786362,6.693364,[-172.78770196948204],[-10.320375656173665],10.320376,172.787702
7,5.452924,277.744424,14.854145,-317.173763,6.030562,[-168.38078929502782],[-9.344661125438106],9.344661,168.380789
8,5.429839,275.256695,14.854145,-314.098767,5.510365,[-166.9883872372335],[-8.827521501820424],8.827522,166.988387
9,5.421592,274.371038,14.854145,-313.004993,5.334607,[-165.03737379031145],[-8.500078866168613],8.500079,165.037374
10,5.413042,273.454503,14.854145,-311.873615,5.157754,[-163.62042123698683],[-8.232341422882024],8.232341,163.620421


In [21]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
